In [ ]:
! pip install -r requirements.txt

### 1. 產生key pair 

In [ ]:
from joserfc import jwk
import os

In [ ]:
if not os.path.exists("private_key.pem"):
    key = jwk.generate_key("RSA", 2048)
    with open("private_key.pem", "wb") as f:
        f.write(key.as_pem())
else:
    with open("private_key.pem", "rb") as f:
        key = jwk.RSAKey.import_key(f.read())

In [ ]:
import json

# Public key with kid and alg
public_key = key.as_dict(private=False)
public_key["kid"] = key.thumbprint()
public_key["alg"] = "RS256"
print(json.dumps(public_key, indent=4))

把以上 Public key 貼到 SMART Launcher "Client Registration & Validation" 頁面的 JWKS 欄位 ，內容會長得跟下面的輸出一模一樣  
p.s. Client Identity Validation 記得設為 Strict

In [ ]:
jwks = {"keys":[public_key]}
print(json.dumps(jwks, indent=4))

In [ ]:
# Private key with kid and alg
private_key = key.as_dict()
private_key["kid"] = key.thumbprint()
private_key["alg"] = "RS256"
print(json.dumps(private_key, indent=4))

### 2. Retrieve .well-known/smart-configuration

In [ ]:
import requests

# 把 Server's FHIR Base URL 貼過來
FHIR_BASE_URL = "https://launch.smarthealthit.org/v/r4/sim/.../fhir"
Client_ID = "client-id"

In [ ]:
resp = requests.get(f"{FHIR_BASE_URL}/.well-known/smart-configuration")
smart_configuration = resp.json()
smart_configuration

### 3. Create Client Assertion Token

In [ ]:
from joserfc import jwt
import datetime

JWT 標頭（Header）欄位說明

| 欄位 | 是否必填 | 說明 |
| :-- | :-- | :-- |
| alg | 必填 | 使用的簽章演算法，例如 `RS384` 或 `ES384`。這用來指定 JWT 是用哪一種演算法簽署的。 |
| kid | 必填 | Key ID — 簽署此 JWT 所使用的金鑰對 (key-pair) 的識別碼。這個 ID 必須在該 client 的 JWK Set 裡是唯一的。 |
| typ | 必填 | 固定值：`JWT`，代表這筆資料是 JSON Web Token。 |

JWT 主體（Claims）欄位說明

| 欄位 | 是否必填 | 說明 |
| :-- | :-- | :-- |
| iss | 必填 | JWT 的發行者（issuer）。此值等於該 client 在註冊授權伺服器時的 `client_id`。也是下方 `sub` 欄位的相同值。 |
| sub | 必填 | JWT 主體（subject），同樣是 `client_id`。這表示 JWT 是代表這個 client 身份。 |
| aud | 必填 | 接收者（audience），即 FHIR 授權伺服器的「token URL」（也就是 JWT 將會被 POST 的 URL）。 |
| exp | 必填 | JWT 的到期時間（epoch 秒）。規範要求此時間必須是「不超過 5 分鐘後」。換句話說，JWT 只能在生成後五分鐘內有效。 |
| jti | 必填 | 一個唯一的隨機字串（nonce），用來識別這筆 JWT，防止重播攻擊。

In [ ]:
header = {
    "alg": "RS256",
    "typ": "JWT",
    "kid": key.thumbprint(),
}

now = datetime.datetime.now()
claims = {
    "iss": Client_ID,
    "sub": Client_ID,
    "aud": smart_configuration['token_endpoint'],
    "exp": now + datetime.timedelta(minutes=5),
    "jti": "random-non-reusable-jwt-id-123"
}

client_assertion = jwt.encode(header, claims, key)
print(client_assertion)

### 4. Retrieve Access Token

In [ ]:
resp = requests.post(
    smart_configuration['token_endpoint'],
    data={
        "grant_type": "client_credentials",
        "client_assertion_type": "urn:ietf:params:oauth:client-assertion-type:jwt-bearer", # 固定值
        "client_assertion": client_assertion,
        "scope": "system/*.*"
    },
    headers={"Content-Type": "application/x-www-form-urlencoded"}
)

token = resp.json().get("access_token")

resp.json()

### 5. Access FHIR Resource

In [ ]:
resp = requests.get(
    f"{FHIR_BASE_URL}/Patient",
    headers={
        "Authorization": f"Bearer {token}"
    }
)
resp.json()